In [190]:
import numpy as np
import pandas as pd

In [191]:
fname = "resp.tsv"
dataset = pd.read_csv(fname, sep="\t")
X = dataset.iloc[:, :].values
X

array([['ResponseCode', 'N 4', 'Response code sent from the MCP server'],
       ['ResponseDesc', 'Strg',
        'Response code description or the response message'],
       ['Trans Id', 'XN 20',
        'Identifier code for the transaction This can be used later to refer to the details for the transaction. ‘X’ represents the prefixed character that indicates type of the transaction message. See Appendix F: Transaction Type Codes for details.'],
       ['ARN', 'Strg 20', 'Acquirer’s Reference Number'],
       ['CustomerId', 'AN 20', 'Customer Identification Number'],
       ['CardNumber', 'N 19',
        'Card number The unmasked card number is included in the Response only if the SendClearCard value in the Request is Y. If the SendClearCard value is N, the Response contains the Card Number in a masked format. If no value is provided for SendClearCard, no card number is provided in the Response.'],
       ['ReferenceID', 'AN 40',
        'A 12 or 15 digit card reference number which u

In [192]:
count = len(X[0, :])
if count == 5:
    isRequst = True
else:
    isRequst = False

In [193]:
mobj = fname[0:fname.index(".")]
main_name = mobj

In [194]:
c = mobj[0].capitalize()
mobj = c + mobj[1:]
mobj
if isRequst:
    mobj += "Type"
else:
    mobj += "ResponseType"

In [195]:
mobj

'RespResponseType'

In [196]:
result = f"""
    {mobj}:
      type: object
      properties:
"""

In [197]:
print(result)


    RespResponseType:
      type: object
      properties:



In [198]:
result += f"""        {main_name}:
          type: object
          properties: 
"""

In [199]:
print(result)


    RespResponseType:
      type: object
      properties:
        resp:
          type: object
          properties: 



In [200]:
def camel_case(string):
  c = string[0].lower()
  string = c + string[1:]
  return string

In [201]:
# def required(string):
#   string = string.split(">")
#   string = [s.strip() for s in string if s != '']
#   return string

In [202]:
# string = "Hello > sf >"
# lst = required(string)
# lst = [camel_case(el) for el in lst]
# lst

In [203]:
def spaces(string):
  return ''.join(['  ' for s in range(string.count('>'))])

In [204]:
def extractNandM(string:str):
  n = ""
  m = ""
  nfound = True
  ffound = False
  for el in string:
    if el.isdigit():
      ffound = True
      if nfound:
        n += el
      else:
        m += el
    else:
      if ffound:
        nfound = False
  return [n, m]
s = "N10"
extractNandM(s)

['10', '']

In [205]:
def makeTSpaces(num, mCount=0, mmCount=0):
  if mmCount > 1 and mCount != 0:
    num += mmCount 
  return ''.join(['  ']*num)

In [206]:
def checkDefault(default):
  if type(default) != float:
    return f"""default: {default}"""
  else:
    return ""

In [207]:
# from cmath import nan
# from dataDictionary import makeDataDic

# def makeObject(lst, obj):
#   tag = lst[0]
#   data = lst[1].replace(" ", "")
#   default = lst[2]
#   req = lst[3].strip(' ')
#   des = lst[-1]
#   n, m = extractNandM(data)
#   dataDic = makeDataDic(n,m, makeTSpaces(7))
#   obj = """"""
#   if tag.count('>') == 0:
#     obj += f"""{makeTSpaces(6)}{camel_case(tag)}:
# {makeTSpaces(7)}{dataDic[data]}
# {makeTSpaces(7)}{checkDefault(default)}
# {makeTSpaces(7)}description: >
# {makeTSpaces(8)}{des}"""
#     return obj
#     if tag.count('>') > 0:
#       pass

In [208]:
# l = ["request", "N10", "N/A", "hello everyone"]
# o = makeObject(l, acquirerObj)
# print(o)

In [209]:
# n = len(X)
# flagA = True
# flagC = True
# flagCA = True
# acquirerObj = f"""    Acquirer_{main_name.capitalize()}:
#       type: object
#       properties:
# """
# for i in range(n):
#   first = camel_case(required(tag[i])[0])
#   acquirerObj += makeObject(X[i], acquirerObj)
#   if first == "acquirer": 
#     if flagA:
#       result += f"""            acquirer:
#               $ref: '#/components/schemas/Acquirer_{main_name.capitalize()}'
# """
#       flagA = False

In [210]:
def ismatchany(pre, curr):
    i = 0
    mCount = 0
    tCount = len(pre) if len(pre) > len(curr) else len(curr)
    while i < len(pre) and i < len(curr):
      if pre[i] == curr[i]:
        mCount += 1
      else:
        break
      i += 1
    return [True if mCount > 0 else False, mCount, tCount - mCount]

In [211]:
from re import sub
def camel_case(s):
  return ''.join([s[0].lower(), s[1:]])

In [212]:
import os
# oCount = 2
# pCount = 0
from dataDictionary import makeDataDic

def makeObject(lst, pre, dSpaces=2):
  obj = """"""
  tag = lst[0]
  data = lst[1].replace(' ', '')
  if isRequst:
    default = lst[2]
    req = lst[3]
  else:
    default = None
    req = None
  des = lst[-1]
  tag = tag.split('>')
  tag = [el.replace(' ', '') for el in tag]
  tag = [camel_case(el) for el in tag]
  tLength = len(tag)
  rLength = 0
  n, m = extractNandM(data)

  ismatch, mCount, mmCount = ismatchany(pre, tag)

  mnum = mCount
  flag = False
  if ismatch:
    tag = tag[mCount:]
    flag = True
  
  length = len(tag)
  if tLength == 1:
    spaces = dSpaces
    obj += f"""{makeTSpaces(spaces, mnum, mmCount)}{tag[0].lower() if len(tag[0]) < 4 else tag[0]}:\n"""
    obj += f"""{makeTSpaces(spaces+1, mnum, mmCount)}type: object\n"""
    obj += f"""{makeTSpaces(spaces+1, mnum, mmCount)}properties:\n"""

  rLength = tLength - length
  if rLength < 0:
    rLength = 0
  i=0
  if mCount > 0:
    mCount -= 2 
  spaces = dSpaces+rLength+mCount
  while i < length-1:
    spaces = dSpaces+rLength+i+mCount
    obj += f"""{makeTSpaces(spaces, mnum, mmCount)}{tag[i].lower() if len(tag[i]) < 4 else tag[i]}:\n"""
    obj += f"""{makeTSpaces(spaces+1, mnum, mmCount)}type: object\n"""
    obj += f"""{makeTSpaces(spaces+1, mnum, mmCount)}properties:\n"""
    i += 1

  obj += f"""{makeTSpaces(spaces+1+1, mnum, mmCount)}{tag[i].lower() if len(tag[i]) < 4 else tag[i]}:\n"""
  obj += f"""{makeTSpaces(spaces+2+1, mnum, mmCount)}{makeDataDic(n, m, makeTSpaces(spaces+2+1, mnum, mmCount))[data]}\n"""
  default = checkDefault(default)
  if default:
    obj += f"""{makeTSpaces(spaces+2+1, mnum, mmCount)}{default}\n"""
  obj += f"""{makeTSpaces(spaces+2+1, mnum, mmCount)}description: >
{makeTSpaces(spaces+4, mnum, mmCount)}{des}\n"""
  return obj

pre = []
if isRequst:
  dSpaces = 2
else:
  dSpaces = 4
bigObj = """"""
for row in X:
  curr = row
  obj = makeObject(curr, pre, dSpaces)
  bigObj += obj
  pre = curr[0].replace(" ", "").split('>')
  pre = [camel_case(el) for el in pre]
bigObj = os.linesep.join([s for s in bigObj.splitlines() if s])
print(bigObj)

        responseCode:
          type: object
          properties:
            responseCode:
              type: string
              minLength: 1
              maxLength: 4
              default: None
              description: >
                Response code sent from the MCP server
        responseDesc:
          type: object
          properties:
            responseDesc:
              type: string
              minLength: 1
              maxLength: 
              default: None
              description: >
                Response code description or the response message
        transId:
          type: object
          properties:
            transId:
              type: string
              format: XN20
              default: None
              description: >
                Identifier code for the transaction This can be used later to refer to the details for the transaction. ‘X’ represents the prefixed character that indicates type of the transaction message. See Appendix F: Tr